# 1. import libraries

In [2]:
import pandas as pd
import nltk
from nltk.probability import FreqDist
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import seaborn as sns

from mbtiData import MBTI

# from fileofclass import class

## 2. Data Loading and Understanding

In [3]:
# Load mbti data
mbti_data = pd.read_csv("data/mbti_1.csv")

In [4]:
mbti_data.describe()

,type,posts
count,8675,8675
unique,16,8675
top,INFP,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
freq,1832,1


In [5]:
mbti_data.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


## 3. cleaning the data

In [6]:
mbti_dict = {}
mbti_type = ["ESTJ", "ENTJ", "ESFJ", "ENFJ", "ISTJ", "ISFJ", "INTJ", "INFJ", "ESTP", "ESFP", "ENTP", "ENFP", "ISTP", "ISFP", "INTP", "INFP"]

for mbti in mbti_type:
    mbti_dict[mbti] = MBTI(mbti)

def fill_posts(line):
    mbti_dict[line["type"]].add_posts(line["posts"])

mbti_data.apply(lambda line: fill_posts(line), axis=1)



0       None
1       None
2       None
3       None
4       None
        ... 
8670    None
8671    None
8672    None
8673    None
8674    None
Length: 8675, dtype: object

In [7]:
print(mbti_dict["ESTJ"].get_posts()[0])

'http://www.youtube.com/watch?v=qsXHcwe3krw


# PREDICTION

In [24]:
# create a new dataframe to store the different features: sentiment analysis, avg word length, num unique words
df_predict = pd.DataFrame(mbti_dict.keys(), columns=["Type"])

## sentiment analysis

In [8]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /Users/renakim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/renakim/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/renakim/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [34]:
df_temp = df_predict.loc[df_predict["Type"] == "INTJ"]
df_temp

,Type
6,INTJ


In [35]:
# Sentiment Analysis
sia = SentimentIntensityAnalyzer()

# Return the sentiment score of each post
def get_polarity_scores(type):
    posts = mbti_dict[type].get_posts()

    neg = 0.0
    pos = 0.0
    neu = 0.0
    compound = 0.0

    for post in posts:
        scores = sia.polarity_scores(post)
        neg += scores['neg']
        pos += scores['pos']
        neu += scores['neu']
        compound += scores['compound']

    # return the total polarity scores for each
    return neg, pos, neu, compound

# try it on just one mbti type
# INTJ took 1 min 7.1 sec
df_temp[['Negative', 'Positive', 'Neutral', 'Compound']] = df_temp["Type"].map(get_polarity_scores)

## Apply polarity scoring function and save scores into separate columns
# df_predict[['Negative', 'Positive', 'Neutral', 'Compound']] = df_predict["Type"].map(get_polarity_scores)

ValueError: Columns must be same length as key